<a href="https://colab.research.google.com/github/LillySh/WishList/blob/main/NLP_HW_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Извлечение токенов из документа требует некоторых манипуляций со строками,
выходящих за рамки возможностей метода str.split(), применявшегося в главе 1.
Необходимо отделить знаки препинания от слов, например кавычки в начале и в конце
высказывания. Нужно также разбить сокращения, такие как we’ll, на составля ющие их слова. После идентификации в документе токенов, подлежащих включению в сло-
варь, вам придется воспользоваться регулярными выражениями, чтобы попытаться

объединить в ходе стемминга слова со схожим смыслом. Далее мы сформируем

векторное представление документов — мультимножество слов, и попытаемся вос-
пользоваться этим вектором для усовершенствования распознавания приветствий

по сравнению с приведенным в конце главы 1 вариантом.

In [ ]:
import numpy as np
import pandas as pd

In [ ]:
sentence = "Thomas Jefferson began building Monticello at the age of 26."

1 шаг: Токенизация. Разбиваем предложение на отдельные слова, знаки препинания.

Извлекаем токены из предложения самым постым методом str.split()

In [ ]:
token_sequence = str.split(sentence)

Единственный минус этого способа, это то что split не разделил знак препинания и число 26.

In [ ]:
token_sequence

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26.']

Формируем словарь. В словаре перечислены все уникальные токены, которые необходимо отслеживать.

Список отсортирован таким образом, что цифры идут пред буквами, а прописные буквы перед строчными.

In [ ]:
vocab = sorted(set(token_sequence))

In [ ]:
','.join(vocab)

'26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the'

In [ ]:
num_tokens = len(token_sequence)

In [ ]:
vocab_size = len(vocab)

Создаем пустую таблицу, ширина которой соответствует количеству уникальных словарных терминов, а высота - длине предложения (в нашем случае 10 строк на 10 столбец)

In [ ]:
onehot_vectors = np.zeros((num_tokens, vocab_size), int)

Для каждого слова в предложении помечаем соответствующий столбец в словаре единицей

In [ ]:
for i, word in enumerate(token_sequence):
  onehot_vectors[i, vocab.index(word)] = 1

In [ ]:
' '.join(vocab)

'26. Jefferson Monticello Thomas age at began building of the'

In [ ]:
onehot_vectors

array([[0, 0, 0, 1, 0, 0, 0, 0, 0, 0],
       [0, 1, 0, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 1, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
       [0, 0, 1, 0, 0, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 1, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

Чтобы таблицу было легче воспринимать, используем таблицу DataFrame из бибилиотеки pandas.

Объект DataFrame отслеживает метки для всех столбцов, позволяя пометить
каждый столбец в нашей таблице токеном или словом, которое он представляет.

In [ ]:
pd.DataFrame(onehot_vectors, columns = vocab)

,26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the
0,0,0,0,1,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,1,0,0,0
3,0,0,0,0,0,0,0,1,0,0
4,0,0,1,0,0,0,0,0,0,0
5,0,0,0,0,0,1,0,0,0,0
6,0,0,0,0,0,0,0,0,0,1
7,0,0,0,0,1,0,0,0,0,0
8,0,0,0,0,0,0,0,0,1,0
9,1,0,0,0,0,0,0,0,0,0


Можно сделать еще понятнее, заменив 0 пробелами

In [ ]:
df = pd.DataFrame(onehot_vectors, columns=vocab)
df[df==0] = ''
df

,26.,Jefferson,Monticello,Thomas,age,at,began,building,of,the
0,,,,1,,,,,,
1,,1,,,,,,,,
2,,,,,,,1,,,
3,,,,,,,,1,,
4,,,1,,,,,,,
5,,,,,,1,,,,
6,,,,,,,,,,1
7,,,,,1,,,,,
8,,,,,,,,,1,
9,1,,,,,,,,,


Исходя из этой таблицы мы можем сделать вывод, что предложение состоит из 10 уникальных и неповторяющихся слов.
1 столбец указывает на слово из словаря, которое присутствовало в этой позиции в документе.

Одно из приятных свойств векторного представления слов и табличного представления документов в том, что исходная информация не теряется.

Важность этого действия в том, что мы преобразовали предложение из слов естественного языка в последовательность чисел (векторов). 

Теперь компьютер может читать и производить арифметические действия над этими векторами так же, как и над любыми другими векторами или списками чисел, а значит, эти унитарные векторы могут быть входными данными для любого конвейера NLP, которому нужны подобные векторы.

Однако полученная таблица получилась довольно большой для одного предложения. Поэтому нам надо вычленить из документа его суть, то есть сжать документ до одного вектора вместо огромной таблицы.

Разобьем документ на более короткие смысловые фрагменты, например
предложения. 

Допустим, бóльшая часть смысла предложения может быть получена из самих слов. Проигнорируем порядок слов и их грамматику и перемешаем их вме-
сте в мультимножестве слов — по одному мультимножеству на каждое предложение в документе.

Предложение о Томасе Джефферсоне в виде вектора мультимножества слов будет выглядеть так:

In [ ]:
sentence_bow = {}
for token in sentence.split():
  sentence_bow[token] = 1

In [ ]:
sorted(sentence_bow.items())

[('26.', 1),
 ('Jefferson', 1),
 ('Monticello', 1),
 ('Thomas', 1),
 ('age', 1),
 ('at', 1),
 ('began', 1),
 ('building', 1),
 ('of', 1),
 ('the', 1)]

Такой словарь можно сделать еще более эффективным, если подставить каждое слово в виде целочисленного указателя на место каждого из слов в лексиконе — списке слов, составляющем словарь для конкретного приложения.

Попробуем еще более эффективную форму словаря. Представим данный словарь в табличной форме с помощью pandas.

In [ ]:
df = pd.DataFrame(pd.Series(dict([(token, 1) for token in sentence.split()])), columns=['sent']).T

In [ ]:
df

,Thomas,Jefferson,began,building,Monticello,at,the,age,of,26.
sent,1,1,1,1,1,1,1,1,1,1


Создадим DataFrame из векторов мультимножеств слов.

Добавим еще несколько предложений.

In [ ]:
sentences = "Thomas Jefferson began building Monticello at the age of 26.\n"
sentences += "Construction was done mostly by local masons and carpenters.\n"

In [ ]:
sentences += "He moved into the South Pavilion in 1770.\n"
sentences += "Turning Monticello into a neoclassical masterpiece was Jefferson's obsession"

In [ ]:
corpus = {}

Используем split('\n'), так как добавили этот символ в конец каждого предложения. Поэтому разбиваем по нему.

In [ ]:
for i, sent in enumerate(sentences.split('\n')):
  corpus['sent{}'.format(i)] = dict((tok,1) for tok in sent.split())

In [ ]:
df = pd.DataFrame.from_records(corpus).fillna(0).astype(int).T

Выведем первые 10 токенов

In [ ]:
df[df.columns[:10]]

,Thomas,Jefferson,began,building,Monticello,at,the,age,of,26.
sent0,1,1,1,1,1,1,1,1,1,1
sent1,0,0,0,0,0,0,0,0,0,0
sent2,0,0,0,0,0,0,1,0,0,0
sent3,0,0,0,0,1,0,0,0,0,0


Можно увидеть небольшое пересечение списков слов используемых в этих предложениях. Среди первых семи слов словаря только Monticello встречается более чем в одном предложении. 

Для сравнения или поиска схожих документов необходимо учесть это пересечение в конвейере. Один из способов найти сходства между предложениями — это подсчитать количество пересекающихся токенов с помощью скалярного произведения.

Минипример подсчета пересечений слов для двух ветокров мультимножеств слов.

Из этих результатов понятно, что в sent0 и sent1 использовались совершенно разные слова.

In [ ]:
df = df.T
df.sent0.dot(df.sent1)

0

А тут, что одно из слов использовалось как в sent0, так и в sent2.

In [ ]:
df.sent0.dot(df.sent2)

1

Также одно из слов словаря применялось в обоих предложениях sent0 и sent3.

In [ ]:
df.sent0.dot(df.sent3)

1

Найдем слово, которое встречалось и в sent0, и в sent 3.

In [ ]:
[(k, v) for (k, v) in (df.sent0 & df.sent3).items() if v]

[('Monticello', 1)]

Токенизация с условием того, что для разделения используют другие символы кроме пробела.

Это можно сделать с помощью регулярныз выражений.

In [ ]:
import re
sentence = "Thomas Jefferson began building Monticello at the age of 26."

Разбиваем предложением по пробелам и знакам препинания, которые встречаютс как минимум один раз.

1. [ ] используются для обозначения множества используемых символов.

2. "+" после закрывающей квадратной скобки (]) означает, что соответствующий шаблону текст должен содержать один или несколько символов в квадратных скобках. 

3. \s в классе символов — краткое обозначение заранее
определенного класса символов, включающего в себя все пробельные символы (пробел, Tab).

re.split - разбивает строку, используя регулярное выражение

In [ ]:
tokens = re.split(r'[-\s.,;!?]+', sentence)

In [ ]:
tokens

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26',
 '']

Используя регулярное выражения, усовершенствуем токенизацию, чтобы точка или пробел после 26 не включалась в словарь.

In [ ]:
pattern = re.compile(r"([-\s.,;!?])+")

In [ ]:
sentence = "Thomas Jefferson began builging Monticello at the age of 26."
tokens = pattern.split(sentence)
[x for x in tokens if x and x not in '- \t\n.,;!?']

['Thomas',
 'Jefferson',
 'began',
 'builging',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26']

Таким образом, мы отфильтровали ненужные токены.

**Далее расширим словарь n-граммами.**

N-грамма — это последовательность, содержащая до n элементов, которые были
извлечены из последовательности этих элементов, обычно строки.

Напишем токенизатор 1-грамм (обычная токениазция по словам):


In [ ]:
sentence = "Thomas Jefferson began building Monticello at the age of 26."
pattern = re.compile(r"([-\s.,;!?])+")
tokens = pattern.split(sentence)
tokens = [x for x in tokens if x and x not in '- \t\n.,;!?']
tokens

['Thomas',
 'Jefferson',
 'began',
 'building',
 'Monticello',
 'at',
 'the',
 'age',
 'of',
 '26']

Далее воспользуемся токенизатором n-грамм из модуля nltk. Мы будем использовать параметр два, чтобы сделать последовательность биграмм.

In [ ]:
from nltk.util import ngrams
list(ngrams(tokens, 2))

[('Thomas', 'Jefferson'),
 ('Jefferson', 'began'),
 ('began', 'building'),
 ('building', 'Monticello'),
 ('Monticello', 'at'),
 ('at', 'the'),
 ('the', 'age'),
 ('age', 'of'),
 ('of', '26')]

Биграммы повторяются по тексту реже, чем 1-граммы, поэтому размер словаря будет меньше.

**Стоп-слова в n-граммах.**

Стоп слова могут нести смысл в предложениях, но не всегда.

Для фильтрации стоп-слов при токенизации достаточно составить собственный список. 

Далее при проходе по циклу по списку токенов мы будем игнорировать несколько стоп-слов.

In [ ]:
stop_words = ['a', 'an', 'the', 'on', 'of', 'off', 'this', 'is']
tokens = ['the', 'house', 'is', 'on', 'fire']
tokens_without_stopwords = [x for x in tokens if x not in stop_words]
print(tokens_without_stopwords)

['house', 'fire']


Наиболее полный список стоп-слов представлен в бибилиотеке NLTK.

In [ ]:
import nltk
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('english')
len(stop_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


179

In [ ]:
stop_words[:7]

['i', 'me', 'my', 'myself', 'we', 'our', 'ours']

In [ ]:
[sw for sw in stop_words if len(sw) == 1]

['i', 'a', 's', 't', 'd', 'm', 'o', 'y']

Следующим шагом форматирования словаря является выраванивание регистра.

Приведение к нижнему регистру сделаем с помощью функции .lower(). Можно это сделать и перед токенизации для всего текста.

In [ ]:
tokens = ['House', 'Visitor', 'Center']
normalized_tokens = [x.lower() for x in tokens]
print(normalized_tokens)

['house', 'visitor', 'center']


**Еще одним методом нормализации является стемминг**

Стемминг возвращает слово к первоначальной форме. Реализуем это также с помощью регулярных выражений.

.findall() - используется для поиска всех непересекающихся совпадений в шаблоне.

Алогритм работы такой:

* Если слово оканчивается более чем одним s, основой считается слово, а суффик-
сом — пустая строка.

* Если слово оканчивается одним s, основа — это слово без s, а суффикс — s.

* Если слово не заканчивается на s, основа — это слово, суффикс не возвращается.

In [ ]:
def stem(phrase):
  return ' '.join([re.findall('^(.*ss|.*?)(s)?$',
  word)[0][0].strip("'") for word in phrase.lower().split()])
  
stem('houses')

'house'

In [ ]:
stem("Doctor House's calls")

'doctor house call'

Но данный алгорит не справится с такими случаями, как dishes или heroes.
В бибилотеке NLTK это уже реализовано.

In [ ]:
from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()
' '.join([stemmer.stem(w).strip("'") for w in "dish washer's washed dishes".split()])

'dish washer wash dish'

**Лемматизация - более расширенная нормализация слова до его семантического корня — леммы.**

При наличии информации о связях между значениями различных слов можно связать несколько слов вместе, даже если их написание сильно различается. Подобная

Например, слова chat, chatter, chatty, chatting и chatbot будут рассматриваться одинаково.

В библиотеке NLTK есть такжеспециальные функции для этого.

In [ ]:
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


In [ ]:
lemmatizer = WordNetLemmatizer()
lemmatizer.lemmatize("better")

'better'

Если не задан параметр pos, то Лемматизатор предполагает, что входное слово существительное и может неправильно определить Лемму.

1. Если pos = "a", то прилагательное
2. "n" - существительное

In [ ]:
lemmatizer.lemmatize("better", pos="a")

'good'

In [ ]:
lemmatizer.lemmatize("good", pos="a")

'good'

In [ ]:
lemmatizer.lemmatize("goods", pos="a")

'goods'

In [ ]:
lemmatizer.lemmatize("goods", pos="n")

'good'

In [ ]:
lemmatizer.lemmatize("goodness", pos="n")

'goodness'

In [ ]:
lemmatizer.lemmatize("best", pos="a")

'best'

Стеммер создает связь между goodness и good, чего не сделал Лемматизатор. Стеммер более подвержен ошибкам и сводят к одной основе большее количество слов, тем самым изменяя смысл текста намного сильнее, чем Лемматизатор.

In [ ]:
stemmer.stem('goodness')

'good'

**Анализ тональности**

1 способ: VADER - анализатор тональности на основе правил

In [ ]:
pip install vaderSentiment

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 125 kB 25.6 MB/s 


In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [ ]:
sa = SentimentIntensityAnalyzer() #Содержит словарь токенов и показателей
sa.lexicon

{'$:': -1.5,
 '%)': -0.4,
 '%-)': -1.5,
 '&-:': -0.4,
 '&:': -0.7,
 "( '}{' )": 1.6,
 '(%': -0.9,
 "('-:": 2.2,
 "(':": 2.3,
 '((-:': 2.1,
 '(*': 1.1,
 '(-%': -0.7,
 '(-*': 1.3,
 '(-:': 1.6,
 '(-:0': 2.8,
 '(-:<': -0.4,
 '(-:o': 1.5,
 '(-:O': 1.5,
 '(-:{': -0.1,
 '(-:|>*': 1.9,
 '(-;': 1.3,
 '(-;|': 2.1,
 '(8': 2.6,
 '(:': 2.2,
 '(:0': 2.4,
 '(:<': -0.2,
 '(:o': 2.5,
 '(:O': 2.5,
 '(;': 1.1,
 '(;<': 0.3,
 '(=': 2.2,
 '(?:': 2.1,
 '(^:': 1.5,
 '(^;': 1.5,
 '(^;0': 2.0,
 '(^;o': 1.9,
 '(o:': 1.6,
 ")':": -2.0,
 ")-':": -2.1,
 ')-:': -2.1,
 ')-:<': -2.2,
 ')-:{': -2.1,
 '):': -1.8,
 '):<': -1.9,
 '):{': -2.3,
 ');<': -2.6,
 '*)': 0.6,
 '*-)': 0.3,
 '*-:': 2.1,
 '*-;': 2.4,
 '*:': 1.9,
 '*<|:-)': 1.6,
 '*\\0/*': 2.3,
 '*^:': 1.6,
 ',-:': 1.2,
 "---'-;-{@": 2.3,
 '--<--<@': 2.2,
 '.-:': -1.2,
 '..###-:': -1.7,
 '..###:': -1.9,
 '/-:': -1.3,
 '/:': -1.3,
 '/:<': -1.4,
 '/=': -0.9,
 '/^:': -1.0,
 '/o:': -1.4,
 '0-8': 0.1,
 '0-|': -1.2,
 '0:)': 1.9,
 '0:-)': 1.4,
 '0:-3': 1.5,
 '0:03': 1.9,
 '

Каждому слову соответствуют баллы. Положительные баллы свидетельствуют о позитивной тональности, отрицательные - о негативной.

Фильтруем список токенов по наличию пробела в токене

In [ ]:
[(tok, score) for tok, score in sa.lexicon.items() if " " in tok]

[("( '}{' )", 1.6),
 ("can't stand", -2.0),
 ('fed up', -1.8),
 ('screwed up', -1.5)]

Оцениваем тональность текста в трех направлениях: положительное, негативное и нейтральное. После чего идет общая оценка тональности предложения.

In [ ]:
sa.polarity_scores(text="Python is very readablr and it's great for NLP.")

{'neg': 0.0, 'neu': 0.661, 'pos': 0.339, 'compound': 0.6249}

In [ ]:
sa.polarity_scores(text="Python is not a bad choice for most applications.")

{'neg': 0.0, 'neu': 0.737, 'pos': 0.263, 'compound': 0.431}

Используем данный способ для определения тональности в 3 следующих предложениях.

In [ ]:
corpus = ["Absolutely perfect! Love it! :-) :-) :-)","Horrible! Completely useless. :(","It was OK. Some good and some bad things."]

In [ ]:
for doc in corpus:
  scores = sa.polarity_scores(doc)
  print('{:+}: {}'.format(scores['compound'], doc))

+0.9428: Absolutely perfect! Love it! :-) :-) :-)
-0.8768: Horrible! Completely useless. :(
-0.1531: It was OK. Some good and some bad things.


Как видно результаты вполне соответствуют настоящей тональности предложений.

Минус: VADER просматривает только 7500 слов, которые есть у него в словаре.

**Наивный байесовский классификатор**

Наивная байесовская модель нацелена на поиск в наборе документов ключевых слов, прогнозирующих значение целевой (выходной) переменной. 

Когда предсказываемой целевой переменной является тональность, эта модель ищет предугадывающие такую тональность слова. 

In [ ]:
pip install nlpia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.0 MB 1.2 MB/s 
     |████████████████████████████████| 706 kB 75.6 MB/s 
     |████████████████████████████████| 120 kB 65.8 MB/s 
     |████████████████████████████████| 1.6 MB 46.3 MB/s 
     |████████████████████████████████| 210 kB 61.5 MB/s 
     |████████████████████████████████| 170 kB 38.7 MB/s 
     |████████████████████████████████| 2.2 MB 62.4 MB/s 
     |████████████████████████████████| 82 kB 673 kB/s 


In [ ]:
from nlpia.data.loaders import get_data

/usr/local/lib/python3.7/dist-packages/pugnlp/constants.py:136: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  [datetime.datetime, pd.datetime, pd.Timestamp])
/usr/local/lib/python3.7/dist-packages/pugnlp/constants.py:158: FutureWarning: The pandas.datetime class is deprecated and will be removed from pandas in a future version. Import from datetime module instead.
  MIN_TIMESTAMP = pd.Timestamp(pd.datetime(1677, 9, 22, 0, 12, 44), tz='utc')
/usr/local/lib/python3.7/dist-packages/pugnlp/tutil.py:100: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  np = pd.np
/usr/local/lib/python3.7/dist-packages/pugnlp/util.py:80: FutureWarning: The pandas.np module is deprecated and will be removed from pandas in a future version. Import numpy directly instead
  np = pd.np
/usr/local/lib/python3.7/dist-packages/nlpia

In [ ]:
movies = get_data('hutto_movies')
movies.head().round(2)

,sentiment,text
id,,
1,2.27,The Rock is destined to be the 21st Century's ...
2,3.53,The gorgeously elaborate continuation of ''The...
3,-0.60,Effective but too tepid biopic
4,1.47,If you sometimes like to go to the movies to h...
5,1.73,"Emerges as something rare, an issue movie that..."


In [ ]:
movies.describe().round(2)

,sentiment
count,10605.00
mean,0.00
std,1.92
min,-3.88
25%,-1.77
50%,-0.08
75%,1.83
max,3.94


Далее токенизируем все тексты рецензий, чтобы создать мультимножества слов для каждого из них.

Поместим их в объект DataFrame библиотеки Pandas.

In [ ]:
from nltk.tokenize import casual_tokenize
from collections import Counter
pd.set_option('display.width',75)
bags_of_words = []
for text in movies.text:
  bags_of_words.append(Counter(casual_tokenize(text)))

In [ ]:
df_bows = pd.DataFrame.from_records(bags_of_words)
df_bows = df_bows.fillna(0).astype(int)
df_bows.shape

(10605, 20756)

In [ ]:
df_bows.head()

,The,Rock,is,destined,to,be,the,21st,Century's,new,...,Ill,slummer,Rashomon,dipsticks,Bearable,Staggeringly,’,ve,muttering,dissing
0,1,1,1,1,2,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,2,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,1,0,4,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Теперь есть все данные, необходимые для модели наивного байесовского классификатора, чтобы искать ключевые слова, предсказывающие тональность
текста на естественном языке:

In [ ]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
nb = nb.fit(df_bows, movies.sentiment > 0)
movies['predicted_sentiment'] =\
movies['predicted_sentiment'] = nb.predict_proba(df_bows)[:,1]*8-4

In [ ]:
movies['error'] = (movies.predicted_sentiment - movies.sentiment).abs()
movies.error.mean().round(1)

1.9

In [ ]:
movies['sentiment_ispositive'] = (movies.sentiment > 0).astype(int)
movies['predicted_ispositive'] = (movies.predicted_sentiment > 0).astype(int)
movies['sentiment predicted_sentiment sentiment_ispositive predicted_ispositive'.split()].head(8)

,sentiment,predicted_sentiment,sentiment_ispositive,predicted_ispositive
id,,,,
1,2.266667,2.511515,1,1
2,3.533333,3.999904,1,1
3,-0.600000,-3.655976,0,0
4,1.466667,1.940954,1,1
5,1.733333,3.910373,1,1
6,2.533333,3.995188,1,1
7,2.466667,3.960466,1,1
8,1.266667,-1.918701,1,0


In [ ]:
(movies.predicted_ispositive == movies.sentiment_ispositive).sum() / len(movies)

0.9344648750589345

Теперь классификатор умеет оценивать тексты
по бинарным категориям «отлично» или «ужасно», поэтому при случайном гадании ошибка MAP была бы около 4. Таким образом, наш конвейер справился примерно в два раза эффективнее алгоритма случайного гадания.

In [108]:
products = get_data('hutto_products')
bags_of_words = []
for text in products.text:
  bags_of_words.append(Counter(casual_tokenize(text)))

df_product_bows = pd.DataFrame.from_records(bags_of_words)
df_product_bows = df_product_bows.fillna(0).astype(int)
df_all_bows = df_bows.append(df_product_bows)
df_all_bows.columns

Index(['The', 'Rock', 'is', 'destined', 'to', 'be', 'the', '21st',
       'Century's', 'new',
       ...
       'sligtly', 'owner', '81', 'defectively', 'warrranty', 'expire',
       'expired', 'voids', 'baghdad', 'harddisk'],
      dtype='object', length=23302)

In [109]:
df_product_bows = df_all_bows.iloc[len(movies):][df_bows.columns]
df_product_bows.shape

(3546, 20756)

In [110]:
df_bows.shape

(10605, 20756)

In [111]:
products['ispos'] = (products.sentiment > 0).astype(int)
products['predicted_ispositive'] = nb.predict(df_product_bows.values).astype(int)
products.head()

/usr/local/lib/python3.7/dist-packages/sklearn/base.py:451: UserWarning: X does not have valid feature names, but MultinomialNB was fitted with feature names
  "X does not have valid feature names, but"


ValueError: ignored

In [113]:
(products.pred == products.ispos).sum() / len(products)

AttributeError: ignored